In [1]:
import sys
sys.path.append('..')

In [2]:
from robustness_analysis.Graph import Graph
from robustness_analysis.Metaweb import Metaweb, DataProcessor
import robustness_analysis.Constants as Constants
from robustness_analysis.AttackStrategy import ThreatenedHabitats
from robustness_analysis.Metaweb import Preparation
from robustness_analysis.Perturbation import Perturbation
import matplotlib.pyplot as plt
import pandas as pd

In [4]:
data_processor = DataProcessor(Constants.ALL_SPECIES_AND_FOOD_GROUPS, Constants.SPECIES_FOR_RANDOMIZED_LINKS)
metaweb = Metaweb(Constants.FOODWEB_02, usecols=[Constants.SOURCE_COL, Constants.TARGET_COL])

In [5]:
metaweb.prepare(Preparation.PREPARE, data_processor)
edge_df = metaweb.get_edges()

In [6]:
graph = Graph(edge_df, source=Constants.SOURCE_COL, target=Constants.TARGET_COL)

In [ ]:
threatened_habitats = ["Grassland", "Forest"]
attack_strategy = ThreatenedHabitats(threatened_habitats)
attack_strategy.create_buckets(graph)

# copy graphs for perturbations

In [ ]:
perturbation = Perturbation(graph)
perturbation.run()

In [ ]:
# Here things get sketchy!!

In [8]:
species_df = pd.read_csv(Constants.ALL_SPECIES_AND_FOOD_GROUPS, usecols=['Species', 'Habitat'])

In [9]:
# processing data frames (they should actually already be preprocessed, before being read by the program!)

edge_df['Source_Name'] = edge_df['Source_Name'].str.lower()
edge_df['Target_Name'] = edge_df['Target_Name'].str.lower()
species_df['Species'] = species_df['Species'].str.lower()
species_df['Habitat'] = species_df['Habitat'].str.lower()

species_df = species_df[species_df['Species'].notna()]

edge_df = edge_df.sort_values(by="Source_Name")
species_df = species_df.sort_values(by="Species")

In [10]:
species_df.head()

,Species,Habitat
23071,aalge,water
6470,abblanda,wetland
8712,abbreviana,wetland
9352,abbreviata,forest
18555,abbreviata,forest; shrubland; urban; wetland


In [12]:
import networkx as nx
G = nx.from_pandas_edgelist(edge_df, source=Constants.SOURCE_COL, target=Constants.TARGET_COL, create_using=nx.DiGraph())

In [13]:
print(len(G.nodes))
print(len(species_df))

21273
23749


In [14]:
sorted(G.nodes)

TypeError: '<' not supported between instances of 'float' and 'str'

In [15]:
species_df['Species']

23071            aalge
6470          abblanda
8712        abbreviana
9352        abbreviata
18555       abbreviata
             ...      
2644       ×versicolor
13915    ×verticillata
13916         ×villosa
6545         ×watereri
7670         ×zeilleri
Name: Species, Length: 23749, dtype: object

In [ ]:
count = 0
for specie in species_df['Species']:
    if specie in G.nodes:
        print(specie)
        count += 1

print(count)

In [ ]:
# Theoretical code for merging information

for specie in species_df['Species']:
    if specie in G.nodes.copy():
        G.nodes[specie]['Habitat'] = specie['Habitat']
    else:
        G.nodes[specie].remove()

strategy = Random()
perturbation = Perturbation(graph, strategy)
perturbation.run()

results = perturbation.get_results()

for key, value in enumerate(results):
    print(key, results)

import matplotlib.pyplot as plt
import numpy as np

metric_names = list(results.keys())
colors = plt.cm.viridis(np.linspace(0, 1, len(metric_names)))

print(colors)

num_rows = len(metric_names)

fig = plt.figure(figsize=(12, 9 * num_rows), dpi=300) 

for i, metric_name in enumerate(metric_names):
    ax = fig.add_subplot(num_rows, 1, i+1)  
    ax.plot(results[metric_name], color=colors[i], linestyle='-', linewidth=2)
    
    ax.set_ylabel(metric_name.replace("_", " ").title(), fontsize=14)
    ax.set_title(f'{metric_name.replace("_", " ").title()} Robustness Trend', fontsize=16)
    
    ax.grid(True, which='both', linestyle='--', linewidth=0.5)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_linewidth(0.5)
    ax.spines['left'].set_linewidth(0.5)

fig.suptitle('Attack Strategy: Random', fontsize=18, y=1.05)
plt.tight_layout()

plt.show()